In [1]:
from google.cloud import bigquery
from google.cloud.bigquery import job
from google.cloud.bigquery import SchemaField
import pandas as pd
import datetime as dt


In [3]:
PROJECT = 'ballosodeuk'
client = bigquery.Client(project=PROJECT)

In [38]:
def estimate_query_cost(client, query):
    job_config = bigquery.QueryJobConfig(
        dry_run=True,
        use_query_cache=False
    )
    query_job = client.query(query, job_config=job_config)
    bytes_processed = query_job.total_bytes_processed
    kb = round(bytes_processed / 1024, 2)
    gb = round(bytes_processed / (1024 ** 3), 2)
    print("예상 쿼리 용량: {} KB, ({} GB)".format(kb, gb))

In [40]:
def run_query(client, query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)
    return query_job

In [54]:
query_name = "0718_유저LTV_생성"
file = f'../query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()

estimate_query_cost(client,query)

예상 쿼리 용량: 177.32 KB, (0.0 GB)


In [53]:

df = run_query(client,query).to_dataframe()
df

,subParam,gmv,commission
0,1eced8e0-0d34-467a-b56b-3b64703bf371,21540.0,970.0
1,4a20e7e0-1429-4f58-bdb2-a549634e8d40,11590.0,523.0
2,b4d35085-865c-409d-afd7-107f2ff2cc19,55420.0,2495.0
3,b70d1ecc-2310-413b-aa0a-c6240368b9f6,0.0,0.0
4,bcfc7195-e991-4646-b2b7-20ecb66beba7,24960.0,1124.0
...,...,...,...
852,4a628023-253c-4b76-b820-bd027beaf603,49760.0,2240.0
853,82985081-dff3-4f56-8938-8d5490913185,25420.0,1144.0
854,acff2a95-0fa7-4a4a-979a-50df518c1c6a,110330.0,4967.0
855,b2e689e8-82c4-4f10-a587-590c2cb3841b,12960.0,584.0


In [45]:
df.query("subParam.isna()")

,subParam,gmv,commission


In [49]:
df.loc[857]

subParam                
gmv           41030359.0
commission     1209165.0
Name: 857, dtype: object